# SBERT-Jittor Evaluation Demo

This notebook loads a pretrained checkpoint and evaluates on STS.


In [ ]:
import os
import sys
from pathlib import Path

def _find_repo_root(start: Path):
    for p in [start] + list(start.parents):
        if (p / 'model' / 'sbert_model.py').is_file():
            return p
    return None

repo_root = _find_repo_root(Path.cwd())
if repo_root is None:
    print('SBERT_JITTOR root not found. Set sys.path manually.')
else:
    sys.path.insert(0, str(repo_root))
    os.chdir(repo_root)
    print(f'Using repo root: {repo_root}')

# If you want to use code from the HF repo instead:
# from huggingface_hub import snapshot_download
# repo_dir = Path(snapshot_download('Kyle-han/roberta-base-nli-mean-tokens'))
# sys.path.insert(0, str(repo_dir))


In [1]:
import jittor as jt
from jittor.dataset import DataLoader
from transformers import AutoTokenizer

from model.sbert_model import SBERTJittor
from utils.download_data import download_sts_benchmark
from utils.data_loader import prepare_sts_dataset, collate_sts
from evaluation.sts.sts_eval_utils import evaluate_sts


[i 0106 00:39:34.299339 72 lock.py:85] Create lock file:C:\Users\hwoos\.cache\jittor\jt1.3.10\cl\py3.10.19\Windows-10-10.x5c\IntelRCoreTM52xf0\b274\jittor.lock
[i 0106 00:39:34.328331 72 compiler.py:956] Jittor(1.3.10.0) src: c:\users\hwoos\miniconda3\envs\ann_final\lib\site-packages\jittor
[i 0106 00:39:35.443513 72 compiler.py:957] cl at C:\Users\hwoos\.cache\jittor\msvc\VC\_\_\_\_\_\bin\cl.exe(19.29.30133)
[i 0106 00:39:35.444698 72 compiler.py:958] cache_path: C:\Users\hwoos\.cache\jittor\jt1.3.10\cl\py3.10.19\Windows-10-10.x5c\IntelRCoreTM52xf0\b274\default
[i 0106 00:39:36.538503 72 __init__.py:412] Found addr2line(2.45) at C:\msys64\mingw64\bin\addr2line.EXE.
[i 0106 00:39:36.540028 72 compiler.py:34] Create cache dir: C:\Users\hwoos\.cache\jittor\jt1.3.10\cl\py3.10.19\Windows-10-10.x5c\IntelRCoreTM52xf0\b274\default\jit
[i 0106 00:39:36.542136 72 compiler.py:34] Create cache dir: C:\Users\hwoos\.cache\jittor\jt1.3.10\cl\py3.10.19\Windows-10-10.x5c\IntelRCoreTM52xf0\b274\default

data.cc
C:\Users\hwoos\.cache\jittor\jt1.3.10\cl\py3.10.19\Windows-10-10.x5c\IntelRCoreTM52xf0\b274\default\data.cc(418): warning C4273: 'jittor::tflag_count': inconsistent dll linkage
c:\users\hwoos\miniconda3\envs\ann_final\lib\site-packages\jittor\src\node.h(88): note: see previous definition of 'tflag_count'
C:\Users\hwoos\.cache\jittor\jt1.3.10\cl\py3.10.19\Windows-10-10.x5c\IntelRCoreTM52xf0\b274\default\data.cc(418): warning C4273: 'jittor::nt': inconsistent dll linkage
c:\users\hwoos\miniconda3\envs\ann_final\lib\site-packages\jittor\src\node.h(18): note: see previous definition of 'nt'
C:\Users\hwoos\.cache\jittor\jt1.3.10\cl\py3.10.19\Windows-10-10.x5c\IntelRCoreTM52xf0\b274\default\data.cc(419): warning C4273: 'jittor::lived_nodes': inconsistent dll linkage
c:\users\hwoos\miniconda3\envs\ann_final\lib\site-packages\jittor\src\node.h(15): note: see previous definition of 'lived_nodes'
C:\Users\hwoos\.cache\jittor\jt1.3.10\cl\py3.10.19\Windows-10-10.x5c\IntelRCoreTM52xf0\b274\

Exception: Run cmd failed: "C:\Users\hwoos\.cache\jittor\msvc\VC\_\_\_\_\_\bin\cl.exe"    -std:c++17         -EHa -MD -utf-8  -nologo -I"C:\Users\hwoos\.cache\jittor\msvc\VC\include" -I"C:\Users\hwoos\.cache\jittor\msvc\win10_kits\include\ucrt" -I"C:\Users\hwoos\.cache\jittor\msvc\win10_kits\include\shared" -I"C:\Users\hwoos\.cache\jittor\msvc\win10_kits\include\um" -DNOMINMAX   -I"c:\users\hwoos\miniconda3\envs\ann_final\lib\site-packages\jittor\src"  -I"c:\users\hwoos\miniconda3\envs\ann_final\lib\site-packages\jittor\extern" -I"c:\users\hwoos\miniconda3\envs\ann_final\include" -I"C:\Users\hwoos\.cache\jittor\jt1.3.10\cl\py3.10.19\Windows-10-10.x5c\IntelRCoreTM52xf0\b274\default"  -O2  -FI "c:\users\hwoos\miniconda3\envs\ann_final\lib\site-packages\jittor\src\utils\vdp" "C:\Users\hwoos\.cache\jittor\jt1.3.10\cl\py3.10.19\Windows-10-10.x5c\IntelRCoreTM52xf0\b274\default\data.cc" -c -Fo: "C:\Users\hwoos\.cache\jittor\jt1.3.10\cl\py3.10.19\Windows-10-10.x5c\IntelRCoreTM52xf0\b274\defaul

In [ ]:
data_dir = '../data'
download_sts_benchmark(data_dir)  # run once


In [ ]:
model, tokenizer, repo_dir = SBERTJittor.from_pretrained(
    'Kyle-han/roberta-base-nli-mean-tokens',
    return_tokenizer=True,
)
if jt.has_cuda:
    jt.flags.use_cuda = 1


In [ ]:
sts_dataset = prepare_sts_dataset(
    data_dir=data_dir,
    dataset_name='STS-B',
    split='test',
    tokenizer=tokenizer,
    max_length=128,
    cache_dir=None,
    overwrite_cache=False,
    tokenize_batch_size=1024,
)

sts_loader = DataLoader(
    sts_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_sts,
)

results = evaluate_sts(model=model, dataloader=sts_loader)
results
